# Split Subjective Days

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import seaborn as sns
import pylab as py
import statistics

# Summary Filename

In [4]:
# Read Spider FRP CSV file
summary_filename = 'Metazygia wittfeldae summary'
summary_df = pd.read_csv(summary_filename + '.csv')

# Get FRP Value of Specific Spider

In [12]:
# Extract FRP values for each spider and create new CSV file

def get_frp(summary_df, spider_name):
    # Creating new FRP CSV file from summary CSV
    summary_df = summary_df[(summary_df['Conditions'] == 'DD') & (summary_df['Spider ID'].str.match('Monitor 1'))]
    frp_df = summary_df[['Spider ID', 'LombSc period']]
    frp_df = frp_df.reset_index(drop = True)
    frp_df = frp_df.set_index('Spider ID')
    frp_df.to_csv(summary_filename + '_period')
    
    # Get FRP value of specific spider
    frp = float(frp_df.values[frp_df.index.str.endswith(spider_name)])
    
    return frp

frp = get_frp(summary_df, 'Monitor 1 Spider 1')

# Activity Filename

In [8]:
# Read spider activity CSV file
filename = 'Metazygia wittfeldae Monitor 1 Updated_DD_binary'
df = pd.read_csv(filename + '.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
#display(df)

# Split Subjective Days Function

Given arguments dataframe in DateTimeIndex, the initial day where the lights didn't turn on, and the FRP of the spider, returns the subjective days of the spider.

In [28]:
def split_subjective_days(df, frp):
    # Get day where lights would turn on but they don't (DD)
    df_start = df.at_time('07:00:00')
    idx = df_start.index[0]
    
    # Subjective days starting from lights not being turned on to given FRP
    dti = pd.date_range(idx, periods = 10, freq = frp)
    dti = dti.round(freq = '1T')
    
    return dti

dti = split_subjective_days(df, str(frp) + 'H')
dti

DatetimeIndex(['2017-04-26 07:00:00', '2017-04-27 05:01:00',
               '2017-04-28 03:01:00', '2017-04-29 01:02:00',
               '2017-04-29 23:03:00', '2017-04-30 21:03:00',
               '2017-05-01 19:04:00', '2017-05-02 17:05:00',
               '2017-05-03 15:05:00', '2017-05-04 13:06:00'],
              dtype='datetime64[ns]', freq=None)

# Extracting Activity

In [16]:
def extract_activity_from_subjective_day(dti, start_index, end_index, column_name):
    # Extract activity based on subjective day index
    activitydf = df.loc[dti[start_index] : dti[end_index], [column_name]]
    
    return activitydf

activitydf = extract_activity_from_subjective_day(dti, 0, 1, 'Monitor 1 Spider 1')

# Get Probability

In [19]:
# This function divides a day into desired number of intervals
def date_range(start, end, intv):
    from datetime import datetime
    start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
    end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    diff = (end  - start ) / intv
    #diff.round(freq = '1T')
    for i in range(intv):
        yield (start + diff * i).strftime("%Y-%m-%d %H:%M:%S")
    yield end.strftime("%Y-%m-%d %H:%M:%S")

In [49]:
def probability_in_multiple_sections(activitydf, start_index, end_index, column_name):
    # Create list of date-times for each subjective hour
    start = str(dti[start_index])
    end = str(dti[end_index])
    mylist = list(date_range(start, end, 24))
    
    list_start = 0
    list_end = len(mylist) - 1
    
    # Loop through list of each subjective hour and return probability of movement
    for list_start, list_end in enumerate(mylist):
        if list_start >= 0 and list_start + 1 >= len(mylist):
            break
        else:
            sub_period = activitydf.loc[mylist[list_start] : mylist[list_start + 1]]
            rows = len(sub_period.index)
            total_period_movement = sub_period.sum()
            prob_period = total_period_movement / rows
            print(prob_period)
        
    return

probability_in_multiple_sections(activitydf, 0, 1, 'Monitor 1 Spider 1')

Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.345455
dtype: float64
Monitor 1 Spider 1    1.0
dtype: float64
Monitor 1 Spider 1    0.327273
dtype: float64
Monitor 1 Spider 1    0.727273
dtype: float64
Monitor 1 Spider 1    0.345455
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.0
dtype: float64
Monitor 1 Spider 1    0.236364
dtype: float64
Monitor 1 Spider 1    0.589286
d

In [53]:
def loop_through_entire_trial(dti):
    start_index = 0
    end_index = start_index + 1
    
    for start_index, end_index in enumerate(dti):
        print(start_index)
        print(end_index)
        print(probability_in_multiple_sections(activitydf, start_index, end_index, 'Monitor 1 Spider 1'))
        #start_index += 1
        
    return

loop_through_entire_trial(dti)

0
2017-04-26 07:00:00


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [17]:
def probability_in_sections(activitydf, start_index, end_index, column_name):
    list_of_dates = extract_activity_from_subjective_day(dti, start_index, end_index, column_name).index.tolist()
    median_date = statistics.median_low(list_of_dates)
    #activitydf = extract_activity_from_subjective_day(dti, start_index, end_index, column_name)
    
    sub_day = activitydf.loc[dti[start_index] : median_date]
    rows = len(sub_day.index)
    total_day_movement = sub_day.sum()
    prob_day = total_day_movement / rows
    
    sub_night = activitydf.loc[median_date : dti[end_index]]
    rows = len(sub_night.index)
    total_night_movement = sub_night.sum()
    prob_night = total_night_movement / rows
    
    return prob_day, prob_night

In [18]:
probability_in_sections(activitydf, 0, 1, 'Monitor 1 Spider 1')

(Monitor 1 Spider 1    0.028744
 dtype: float64,
 Monitor 1 Spider 1    0.270393
 dtype: float64)

# Trying to Loop

# Testing

In [60]:
df_start = df.at_time('07:00:00')
idx = df_start.index[0]
    
# Subjective days starting from lights not being turned on to given FRP
dti = pd.date_range(idx, periods = 10, freq = '22.0111503H')
dti = dti.round(freq = '1T')
#print(dti)

start_index = 0
end_index = 1

list_of_dates = extract_activity_from_subjective_day(dti, start_index, end_index, 'Monitor 1 Spider 1').index.tolist()
median_date = statistics.median_low(list_of_dates)

activitydf = df.loc[dti[start_index] : dti[end_index], ['Monitor 1 Spider 1']]
sub_day = activitydf.loc[dti[start_index] : median_date]
#display(sub_day)
rows = len(sub_day.index)
total_day_movement = sub_day.sum()
prob_day = total_day_movement / rows
print(prob_day)
    
sub_night = activitydf.loc[median_date : dti[end_index]]
rows = len(sub_night.index)
total_night_movement = sub_night.sum()
prob_night = total_night_movement / rows
print(prob_night)

start_index = 0
end_index = 1

for start_index, end_index in enumerate(dti):
    list_of_dates = extract_activity_from_subjective_day(dti, start_index, end_index, 'Monitor 1 Spider 1').index.tolist()
    median_date = statistics.median_low(list_of_dates)

    activitydf = df.loc[dti[start_index] : dti[end_index], ['Monitor 1 Spider 1']]
    sub_day = activitydf.loc[dti[start_index] : median_date]
    display(sub_day)
    rows = len(sub_day.index)
    total_day_movement = sub_day.sum()
    prob_day = total_day_movement / rows
    print(prob_day)
    
    sub_night = activitydf.loc[median_date : dti[end_index]]
    rows = len(sub_night.index)
    total_night_movement = sub_night.sum()
    prob_night = total_night_movement / rows
    print(prob_night)

Monitor 1 Spider 1    0.028744
dtype: float64
Monitor 1 Spider 1    0.270393
dtype: float64


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices